# Installation

```python
pip install "mmsegmentation>=1.0.0"
```

## Verify

**Step 1:** We need to download config and checkpoint files.
```bash
mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest ./models
```

**Step 2:** Verify the inference demo.

In [2]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv

config_file = 'models/pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'
checkpoint_file = 'models/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img = 'demo/demo.png'  # or img = mmcv.imread(img), which will only load it once
result = inference_model(model, img)
# visualize the results in a new window
show_result_pyplot(model, img, result, show=True)
# or save the visualization results to image files
# you can change the opacity of the painted segmentation map in (0, 1].
show_result_pyplot(model, img, result, show=True, out_file='outputs/result.jpg', opacity=0.5)
# test a video and show the results
# video = mmcv.VideoReader('video.mp4')
# for frame in video:
#   result = inference_model(model, frame)
#   show_result_pyplot(model, frame, result, wait_time=1)

Loads checkpoint by local backend from path: models/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth


array([[[ 87,  94,  87],
        [ 69,  80,  74],
        [ 94,  99,  99],
        ...,
        [ 86,  90,  80],
        [ 88,  91,  81],
        [ 86,  90,  80]],

       [[ 89,  96,  88],
        [ 73,  83,  76],
        [ 94,  99,  99],
        ...,
        [ 86,  90,  80],
        [ 88,  90,  79],
        [ 86,  90,  80]],

       [[ 87,  94,  87],
        [ 73,  83,  76],
        [ 94,  99,  99],
        ...,
        [ 86,  90,  80],
        [ 86,  88,  78],
        [ 86,  90,  80]],

       ...,

       [[ 88,  63,  91],
        [ 88,  63,  91],
        [ 88,  61,  90],
        ...,
        [102,  77, 102],
        [100,  76, 101],
        [102,  77, 102]],

       [[ 88,  61,  90],
        [ 88,  63,  91],
        [ 86,  61,  89],
        ...,
        [ 99,  73, 100],
        [100,  74, 100],
        [100,  74, 101]],

       [[ 88,  61,  90],
        [ 88,  63,  91],
        [ 86,  61,  89],
        ...,
        [ 90,  60,  88],
        [ 89,  62,  89],
        [ 92,  65,  92]]

In [4]:
import cv2
import mmcv
import mmengine
import numpy as np

# 1. Video Ayarları
input_video_path = 'demo/demo.mp4'  
output_video_path = 'outputs/video_result.mp4'

# 2. Videoyu Yükle
video = mmcv.VideoReader(input_video_path)
print(f"Video işleniyor: {len(video)} kare, {video.fps} FPS")

# 3. Video Yazıcıyı Hazırla
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = None

# mmcv yerine mmengine kullanıyoruz
prog_bar = mmengine.ProgressBar(len(video))

# 4. Kare Kare İşleme Döngüsü
for frame in video:
    # Inference yap
    result = inference_model(model, frame)
    
    # Sonucu görselleştir (show=False önemli)
    vis_frame = show_result_pyplot(model, frame, result, show=False)
    
    # Video yazıcıyı ilk karede başlat
    if video_writer is None:
        height, width, _ = vis_frame.shape
        video_writer = cv2.VideoWriter(output_video_path, fourcc, video.fps, (width, height))
    
    # Kareyi dosyaya yaz
    video_writer.write(vis_frame)
    prog_bar.update()

# 5. Kaydet ve Temizle
if video_writer:
    video_writer.release()

print(f'\nTamamlandı! Video şuraya kaydedildi: {output_video_path}')

Video işleniyor: 67 kare, 30.0 FPS
[                                                  ] 0/67, elapsed: 0s, ETA:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 67/67, 0.6 task/s, elapsed: 120s, ETA:     0s
Tamamlandı! Video şuraya kaydedildi: outputs/video_result.mp4
